# 応用情報工学演習：Scikit-Learnマスターコース
## 実践編 配布用コードファイル

#### 基本ライブラリの読み込み

In [1]:
import os, sys
import glob

import warnings
warnings.simplefilter('ignore')

import numpy as np
from sklearn.metrics import accuracy_score

from PIL import Image

np.random.seed(1)

#### データ読み込み関数

In [2]:
#-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-#
#                               このセルは変更を禁止します                                  #
#-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-#
#-------------------------------------------------------------------------------------------#
#-                                  データ読み込み関数                                     -#
#-------------------------------------------------------------------------------------------#
def load_data_(root_dir='../data/gw_dataset'):
    """
    グループワーク用　学習用データ読み込み関数

    Args:
        * root_dir: 配布したgw_datasetへのパスを指定する (default: '../data/gw_dataset')
        
    Output:
        * X_train:  正解ラベル付き学習用画像   shape: (140x150,528)
                    1行あたり、画像1枚の画素値を格納したnp.array
                    画像140枚、各画像224x224=150,528次元のベクトル
                    
        * y_train:  X_trainの正解ラベル        shape: (140, )
                    クラス数は7　140枚分のクラスラベル（0～6）の整数が格納されている
        
        * X_trainu: 正解ラベルなし学習用画像   shape: (210x150,528)
                    形式はX_trainと同じ
                    
        * X_val:   正解ラベル付き検証用画像   shape: (70x150,528)
                    形式はX_trainと同じ
                    
        * y_val:    X_valの正解ラベル          shape: (70, )
                    形式はy_valと同じ                             
    """
    
    #----------------- 学習用データの読み込み -------------------#
    train_paths = sorted(glob.glob(os.path.join(root_dir, "train", "*.png")))
    
    X_train = np.array([np.array(Image.open(p)).astype(np.float32).ravel() for p in train_paths])
    y_train = np.load(os.path.join(root_dir, "y_train.npy"))

    #------------ ラベルなし学習用データの読み込み --------------#
    trainu_paths = glob.glob(os.path.join(root_dir, "train-u", "*.png"))
    X_trainu = np.array([np.array(Image.open(p)).astype(np.float32).ravel() for p in trainu_paths])
    
    #----------------- 検証用データの読み込み -------------------#
    val_paths = glob.glob(os.path.join(root_dir,"val", "*.png"))
    X_val = np.array([np.array(Image.open(p)).astype(np.float32).ravel() for p in val_paths])
    y_val = np.load(os.path.join(root_dir, "y_val.npy"))

    return X_train, y_train, X_trainu, X_val, y_val

In [3]:
### データ読み込み関数の実行
X_train, y_train, X_trainu, X_val, y_val = load_data_() # すべて必要な場合
print(X_train.shape, y_train.shape, X_trainu.shape, X_val.shape, y_val.shape)

# X_train, y_train, _, X_val, y_val = load_data_() # いらないデータがある場合
# print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(140, 150528) (140,) (210, 150528) (70, 150528) (70,)


#### 評価関数

In [4]:
#-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-#
#                               このセルは変更を禁止します                                  #
#-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-#
#-------------------------------------------------------------------------------------------#
#-                                       評価関数                                          -#
#-------------------------------------------------------------------------------------------#

def eval_(y_pred):
    """
    グループワーク用　評価関数
    実行する前にy_valがglobalスコープに読み込まれている必要がある

    Args:
        * y_pred:   識別結果　y_valと同形式・同shapeでなければならない
    """

    try:
        y_val
    except NameError as e:
        print("y_valが読み込まれていません")
        
    assert y_pred.shape == y_val.shape, 'y_predとy_valのサイズが一致しません'
    
    print("valデータでの識別精度:{0:.3f}".format(accuracy_score(y_val, y_pred)))

#### 線形SVM(SVC)を使った場合の例

In [8]:
#-------------------------------------------------------------------------------------------#
#-                                線形SVM(SVC)を使った場合の例                                -#
#-------------------------------------------------------------------------------------------#
from sklearn.svm import SVC

clf = SVC(kernel='linear', C=1)# SVCオブジェクトを作成
clf.fit(X_train, y_train)       # 学習の実行

#########################################################################
# y_predに識別結果を格納してeval_(y_pred)を実行するとvalでの精度が評価できる
# y_predはy_valと同形式・同shapeである必要があることに注意

y_pred = clf.predict(X_val)     # X_valの識別結果をy_predに格納
eval_(y_pred)                   # 評価関数の実行

valデータでの識別精度:0.500


In [16]:
#criterion = "gini"
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

for i in range(10,101):
    Rf = RandomForestClassifier(random_state = i)
    Rf.fit(X_train, y_train)

    y_pred = Rf.predict(X_val)
    print(f"i = {i}のとき")
    eval_(y_pred)
#random_state = 25,72のときvalデータでの識別精度:0.586

i = 10のとき
valデータでの識別精度:0.471
i = 11のとき
valデータでの識別精度:0.486
i = 12のとき
valデータでの識別精度:0.457
i = 13のとき
valデータでの識別精度:0.500
i = 14のとき
valデータでの識別精度:0.500
i = 15のとき
valデータでの識別精度:0.514
i = 16のとき
valデータでの識別精度:0.500
i = 17のとき
valデータでの識別精度:0.500
i = 18のとき
valデータでの識別精度:0.471
i = 19のとき
valデータでの識別精度:0.443
i = 20のとき
valデータでの識別精度:0.471
i = 21のとき
valデータでの識別精度:0.486
i = 22のとき
valデータでの識別精度:0.500
i = 23のとき
valデータでの識別精度:0.500
i = 24のとき
valデータでの識別精度:0.514
i = 25のとき
valデータでの識別精度:0.586
i = 26のとき
valデータでの識別精度:0.486
i = 27のとき
valデータでの識別精度:0.471
i = 28のとき
valデータでの識別精度:0.514
i = 29のとき
valデータでの識別精度:0.557
i = 30のとき
valデータでの識別精度:0.514
i = 31のとき
valデータでの識別精度:0.514
i = 32のとき
valデータでの識別精度:0.514
i = 33のとき
valデータでの識別精度:0.543
i = 34のとき
valデータでの識別精度:0.543
i = 35のとき
valデータでの識別精度:0.557
i = 36のとき
valデータでの識別精度:0.486
i = 37のとき
valデータでの識別精度:0.486
i = 38のとき
valデータでの識別精度:0.457
i = 39のとき
valデータでの識別精度:0.529
i = 40のとき
valデータでの識別精度:0.486
i = 41のとき
valデータでの識別精度:0.529
i = 42のとき
valデータでの識別精度:0.471
i = 43のとき
valデータでの識別精度:0.486
i = 44のとき
valデ

In [18]:
#criterion = "entropy"
for i in range(0,101):
    Rf = RandomForestClassifier(random_state = i,
                               criterion = "entropy")
    Rf.fit(X_train, y_train)

    y_pred = Rf.predict(X_val)
    print(f"i = {i}のとき")
    eval_(y_pred)
#random_state = 85のときvalデータでの識別精度:0.600

i = 0のとき
valデータでの識別精度:0.543
i = 1のとき
valデータでの識別精度:0.500
i = 2のとき
valデータでの識別精度:0.543
i = 3のとき
valデータでの識別精度:0.471
i = 4のとき
valデータでの識別精度:0.543
i = 5のとき
valデータでの識別精度:0.557
i = 6のとき
valデータでの識別精度:0.543
i = 7のとき
valデータでの識別精度:0.514
i = 8のとき
valデータでの識別精度:0.486
i = 9のとき
valデータでの識別精度:0.529
i = 10のとき
valデータでの識別精度:0.500
i = 11のとき
valデータでの識別精度:0.514
i = 12のとき
valデータでの識別精度:0.529
i = 13のとき
valデータでの識別精度:0.529
i = 14のとき
valデータでの識別精度:0.543
i = 15のとき
valデータでの識別精度:0.557
i = 16のとき
valデータでの識別精度:0.543
i = 17のとき
valデータでの識別精度:0.543
i = 18のとき
valデータでの識別精度:0.443
i = 19のとき
valデータでの識別精度:0.486
i = 20のとき
valデータでの識別精度:0.471
i = 21のとき
valデータでの識別精度:0.457
i = 22のとき
valデータでの識別精度:0.457
i = 23のとき
valデータでの識別精度:0.529
i = 24のとき
valデータでの識別精度:0.471
i = 25のとき
valデータでの識別精度:0.486
i = 26のとき
valデータでの識別精度:0.471
i = 27のとき
valデータでの識別精度:0.529
i = 28のとき
valデータでの識別精度:0.514
i = 29のとき
valデータでの識別精度:0.514
i = 30のとき
valデータでの識別精度:0.457
i = 31のとき
valデータでの識別精度:0.486
i = 32のとき
valデータでの識別精度:0.586
i = 33のとき
valデータでの識別精度:0.514
i = 34のとき
valデータでの識別精度:0

In [46]:
from sklearn.model_selection import GridSearchCV
import random
import pandas as pd

Rf = RandomForestClassifier()

# 生成されたランダムな数値を格納するリスト
rand = []

# 7つのランダムな数値を生成するループ
for _ in range(7):
    # 1から10000の範囲でランダムな整数を生成し、リストに追加する
    random_number = random.randint(1, 10000)
    rand.append(random_number)

# グリッド探索するハイパーパラメータとその値の範囲を指定（辞書）
parameters = {
    'random_state': rand,
    'n_estimators': [75,100],
    'max_depth': [512,768],
    'criterion': ('gini','entropy','log_loss'),
    'min_samples_split': [2,4],
    'min_samples_leaf': [1,2,3],
    'max_features': ('sqrt','log2')
    'bootstrap': (True,False),
    #'n_jos': [-1],
    
}

# ベース識別器のオブジェクトとハイパーパラメータ辞書を指定して
# グリッド探索用のオブジェクトを作成
gs = GridSearchCV(Rf, parameters)

# fitでグリッド探索開始
gs.fit(X_train, y_train)

# 結果を表示：pandasのテーブルを使うと見やすい
df = pd.DataFrame.from_dict(gs.cv_results_)

best_para = df[df['rank_test_score']==1]
print('一番良いパラメタ')
print(best_para)

     mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0         0.450547      0.040947         0.015659        0.002983   
1         0.468083      0.031009         0.013711        0.003590   
2         0.422912      0.040156         0.015336        0.001468   
3         0.482445      0.052754         0.011086        0.003098   
4         0.458959      0.034070         0.012615        0.002863   
..             ...           ...              ...             ...   
667       0.632231      0.067183         0.015155        0.003625   
668       0.643010      0.034730         0.013595        0.002033   
669       0.617703      0.024009         0.016644        0.003014   
670       0.620581      0.006299         0.015317        0.002751   
671       0.648948      0.040696         0.017148        0.004712   

    param_bootstrap param_criterion param_max_depth param_min_samples_leaf  \
0              True         entropy             512                      1   
1              

In [47]:
# DataFrameを作成
df = pd.DataFrame.from_dict(gs.cv_results_)

# rank_test_scoreが1の行を抽出
best_params = df[df['rank_test_score'] == 1]

print("rank_test_scoreが1の行:")
print(best_params)

rank_test_scoreが1の行:
     mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0         0.450547      0.040947         0.015659        0.002983   
1         0.468083      0.031009         0.013711        0.003590   
2         0.422912      0.040156         0.015336        0.001468   
3         0.482445      0.052754         0.011086        0.003098   
4         0.458959      0.034070         0.012615        0.002863   
..             ...           ...              ...             ...   
667       0.632231      0.067183         0.015155        0.003625   
668       0.643010      0.034730         0.013595        0.002033   
669       0.617703      0.024009         0.016644        0.003014   
670       0.620581      0.006299         0.015317        0.002751   
671       0.648948      0.040696         0.017148        0.004712   

    param_bootstrap param_criterion param_max_depth param_min_samples_leaf  \
0              True         entropy             512                     

In [52]:
from sklearn.model_selection import cross_val_predict
#グリッド探索で得たパラメタの精度を調査
Rf = RandomForestClassifier(random_state = 3900,
                            criterion = "entropy",
                           min_samples_split = 2,
                           min_samples_leaf=1,
                           bootstrap=False,
                           max_depth=768,
                           max_features="log2")

y_pred = cross_val_predict(Rf, X_train, y_train, cv = 10)

# 交差検証の精度を出力
print("交差検証での精度: {:.3f}".format(accuracy_score(y_train, y_pred)))

Rf.fit(X_train, y_train)
y_pred = Rf.predict(X_val)

eval_(y_pred)

交差検証での精度: 0.493
valデータでの識別精度:0.514


In [37]:
max_accu = 0
max_idx = 0
for i in range(0,1001):
    Rf = RandomForestClassifier(random_state = i,
                               criterion = "entropy",
                               max_features = "log2")
    Rf.fit(X_train, y_train)

    y_pred = Rf.predict(X_val)
    #eval_(y_pred)
    if max_accu < accuracy_score(y_val, y_pred):
        max_accu = accuracy_score(y_val, y_pred)
        max_idx = i
        
print(f" randam_state = {max_idx} : max_accu = {max_accu}")
#random_state = 38のときvalデータでの識別精度:0.600

 randam_state = 38 : max_accu = 0.6


In [ ]:
#一番良いのは0.600だった
#criterion = "entropy"
for i in range(0,101):
    Rf = RandomForestClassifier(random_state = i,
                               criterion = "entropy")
    Rf.fit(X_train, y_train)

    y_pred = Rf.predict(X_val)
    print(f"i = {i}のとき")
    eval_(y_pred)
#random_state = 85のときvalデータでの識別精度:0.600